# This notebook is for course Five : Regression (Supervised Learning)

### Importing Data Files

In [1]:
import pandas as pd
import numpy as np

In [4]:
product_data = pd.read_csv('../Student Files/Product Data Set - Student 2 of 3.csv', sep = '|')

In [6]:
customer_data = pd.read_csv('../Student Files/Customer Data Set - Student 1 of 3.csv')

In [9]:
transaction_data = pd.read_csv('../Student Files/Transaction Data Set - Student 3 of 3.csv', sep = '|')

### Changing Data Types

In [10]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : x.replace('$',''))

In [11]:
customer_data['INCOME']=customer_data['INCOME'].map(lambda x : int(x.replace(',','')))

### Creating Customer View

In [13]:
trans_products=transaction_data.merge(product_data,how='inner', left_on='PRODUCT NUM', right_on='PRODUCT CODE')

In [14]:
trans_products['UNIT LIST PRICE']=trans_products['UNIT LIST PRICE'].map(lambda x : float(x.replace('$','')))

In [15]:
trans_products['TOTAL PRICE']=trans_products['QUANTITY PURCHASED'] * trans_products['UNIT LIST PRICE'] * (1- trans_products['DISCOUNT TAKEN'])

In [16]:
customer_prod_categ=trans_products.groupby(['CUSTOMER NUM','PRODUCT CATEGORY']).agg({'TOTAL PRICE':'sum'})

In [17]:
customer_prod_categ=customer_prod_categ.reset_index()

In [18]:
customer_pivot=customer_prod_categ.pivot(index='CUSTOMER NUM',columns='PRODUCT CATEGORY',values='TOTAL PRICE')

In [21]:
trans_total_spend = trans_products.groupby('CUSTOMER NUM').agg({'TOTAL PRICE':'sum'}).\
rename(columns={'TOTAL PRICE':'TOTAL SPENT'})

In [22]:
customer_KPIs=customer_pivot.merge(trans_total_spend,how='inner',left_index=True, right_index=True )

In [23]:
customer_KPIs=customer_KPIs.fillna(0)
customer_KPIs.head()

,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,TOTAL SPENT
CUSTOMER NUM,,,,,
10001,4022.430,1601.315,68.688,1134.337,6826.770
10002,2312.509,2473.163,276.779,0.000,5062.451
10003,2887.382,5414.418,260.640,0.000,8562.440
10004,3637.213,1840.211,45.270,0.000,5522.694
10005,213.512,0.000,0.000,0.000,213.512


In [24]:
customer_all_view=customer_data.merge(customer_KPIs,how='inner', left_on='CUSTOMERID', right_index=True)

In [25]:
customer_all_view.head()

,CUSTOMERID,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,ENROLLMENT DATE,HOUSEHOLD SIZE,MARITAL STATUS,APPAREL,ELECTRONICS,FOOD,HEALTH & BEAUTY,TOTAL SPENT
0,10001,0,64,133498,5,enrolled,06-03-2013,4,Single,4022.430,1601.315,68.688,1134.337,6826.770
1,10002,0,42,94475,9,notenrolled,NaN,6,Married,2312.509,2473.163,276.779,0.000,5062.451
2,10003,0,40,88610,9,enrolled,02-09-2010,5,Married,2887.382,5414.418,260.640,0.000,8562.440
3,10004,0,38,84313,8,enrolled,06-04-2015,1,Single,3637.213,1840.211,45.270,0.000,5522.694
4,10005,0,30,51498,3,notenrolled,NaN,1,Single,213.512,0.000,0.000,0.000,213.512


# Linear Regression 

In [42]:
features = ['GENDER','AGE', 'INCOME', 'EXPERIENCE SCORE', 'LOYALTY GROUP', 'HOUSEHOLD SIZE', 'MARITAL STATUS']
DF_input = customer_all_view[features]

In [43]:
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,enrolled,4,Single
1,0,42,94475,9,notenrolled,6,Married
2,0,40,88610,9,enrolled,5,Married
3,0,38,84313,8,enrolled,1,Single
4,0,30,51498,3,notenrolled,1,Single


In [44]:
pd.get_dummies(DF_input['MARITAL STATUS'])

,Divorced,Married,Single,Widow/Widower
0,0,0,1,0
1,0,1,0,0
2,0,1,0,0
3,0,0,1,0
4,0,0,1,0
...,...,...,...,...
495,0,0,1,0
496,0,0,1,0
497,0,1,0,0
498,0,0,1,0


In [45]:
# We could use a function to encode:
def encode_loyalty(value):
    if value == 'enrolled':
        return 1
    else:
        return 0

In [46]:
DF_input['LOYALTY GROUP'] = DF_input['LOYALTY GROUP'].apply(encode_loyalty)
DF_input.head()

/home/wanjiku/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS
0,0,64,133498,5,1,4,Single
1,0,42,94475,9,0,6,Married
2,0,40,88610,9,1,5,Married
3,0,38,84313,8,1,1,Single
4,0,30,51498,3,0,1,Single


In [47]:
# now for the whole dataframe:
DF_input = pd.get_dummies(DF_input)

In [48]:
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0,64,133498,5,1,4,0,0,1,0
1,0,42,94475,9,0,6,0,1,0,0
2,0,40,88610,9,1,5,0,1,0,0
3,0,38,84313,8,1,1,0,0,1,0
4,0,30,51498,3,0,1,0,0,1,0


In [49]:
# now we scale because of the big differences in values e.g income and household size
from sklearn import preprocessing

In [51]:
DF_input_column_names = DF_input.columns.values
DF_input_column_names

array(['GENDER', 'AGE', 'INCOME', 'EXPERIENCE SCORE', 'LOYALTY GROUP',
       'HOUSEHOLD SIZE', 'MARITAL STATUS_Divorced',
       'MARITAL STATUS_Married', 'MARITAL STATUS_Single',
       'MARITAL STATUS_Widow/Widower'], dtype=object)

In [53]:
DF_input = preprocessing.minmax_scale(DF_input)

In [56]:
DF_input = pd.DataFrame(DF_input, columns = DF_input_column_names)
DF_input.head()

,GENDER,AGE,INCOME,EXPERIENCE SCORE,LOYALTY GROUP,HOUSEHOLD SIZE,MARITAL STATUS_Divorced,MARITAL STATUS_Married,MARITAL STATUS_Single,MARITAL STATUS_Widow/Widower
0,0.0,0.638889,0.872818,0.444444,1.0,0.6,0.0,0.0,1.0,0.0
1,0.0,0.333333,0.572046,0.888889,0.0,1.0,0.0,1.0,0.0,0.0
2,0.0,0.305556,0.526842,0.888889,1.0,0.8,0.0,1.0,0.0,0.0
3,0.0,0.277778,0.493722,0.777778,1.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.166667,0.240799,0.222222,0.0,0.0,0.0,0.0,1.0,0.0


In [57]:
from sklearn.model_selection import train_test_split

In [67]:
# split the input features (DF_input) and target variable(customer_all_view['TOTAL SPENT']) 
# they go in the form of (x,y) i.e. x = input y = target so there's train(x,y) and test(x,y)
X_train,X_test,Y_train,Y_test = \
train_test_split(DF_input, customer_all_view['TOTAL SPENT'], test_size = 0.2, random_state = 42)

In [68]:
from sklearn.linear_model import LinearRegression

In [69]:
# instantiate the model by running:
regr = LinearRegression()

In [70]:
# train the model using the training dataset:
regr.fit(X_train,Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None, normalize=False)

In [75]:
# now that we've trained our model let's predict
predicted = regr.predict(X_test)

In [80]:
# obtain the regression coefficients using the reg.coef_ method:
print("Coefficients: \n", regr.coef_)
# the above coefficients are in the order of:
DF_input_column_names

Coefficients: 
 [   35.58494877 -7076.18001534 12403.0952701    505.84578936
  1566.48092344  1040.19923196   377.33573376   -82.02665324
   -34.57620088  -260.73287964]


array(['GENDER', 'AGE', 'INCOME', 'EXPERIENCE SCORE', 'LOYALTY GROUP',
       'HOUSEHOLD SIZE', 'MARITAL STATUS_Divorced',
       'MARITAL STATUS_Married', 'MARITAL STATUS_Single',
       'MARITAL STATUS_Widow/Widower'], dtype=object)

In [106]:
table1 = pd.DataFrame(DF_input_column_names, columns = ['Predictor variables'])
table2 = pd.DataFrame(regr.coef_, columns = ['Coefficients'])
predictor_coeffs = pd.concat([table1,table2], axis  =1)
predictor_coeffs

,Predictor variables,Coefficients
0,GENDER,35.584949
1,AGE,-7076.180015
2,INCOME,12403.095270
3,EXPERIENCE SCORE,505.845789
4,LOYALTY GROUP,1566.480923
5,HOUSEHOLD SIZE,1040.199232
6,MARITAL STATUS_Divorced,377.335734
7,MARITAL STATUS_Married,-82.026653
8,MARITAL STATUS_Single,-34.576201
9,MARITAL STATUS_Widow/Widower,-260.732880


Positive values indicate a positive correlation between the variable/coefficient and the target variable that is TOTAL SPENT. Negative values indicate that the target variable and the coefficient are negatively correlated.

**The magnitude of a predictor variable can be measured by the magnitude of its regression coefficient. High magnitude = high importance and vice versa.**

We measure using absolute values of the coefficients since the sign(+/-) is just an indication of relationship

In [112]:
abs(predictor_coeffs['Coefficients']).sort_values(ascending = False)

2    12403.095270
1     7076.180015
4     1566.480923
5     1040.199232
3      505.845789
6      377.335734
9      260.732880
7       82.026653
0       35.584949
8       34.576201
Name: Coefficients, dtype: float64

As we can see from the above table, the order of importance is: Income > Age > Loyalty Group > Household Size > Experience Score > Marital Status Divorced > Widow/Widower > Married > Gender > Single

In [113]:
# obtain the constant or intercept by running:
print('intercept: \n', regr.intercept_)

intercept: 
 -38.16947521865768


The linear regression equation to predict total spending is as follows:

Predicted Total Spend = 35.58 * Gender - 7076.18 * AGE + 12403.1 * Income + 505.85 * Experience score + 1566.48 * Loyalty group + 10402 * House hold size + 377.34 * (Marital Status – Divorced) – 82.03 * (Marital Status – Married) - 34.58 * (Marital Status – Single) - 260.73 * (Marital Status – Widow) - 38.17

Evaluate this model by calculating the **mean squared error**.

In [114]:
from sklearn.metrics import mean_squared_error

In [121]:
print(round(mean_squared_error(Y_test, predicted),2) )

3164098.64


In [122]:
# therefore the mean error:
np.sqrt(3164098.64)

1778.7913424570067

This means that our linear regression model predicts total spending with an average error of 1778.79 dollars.

**Here we've used all variables as input variables, ideally you should only use those with relationships that we already saw during the data understanding and prep stage, repeat this process with those whose relationships we had established then check error levels**